<a href="https://colab.research.google.com/github/Anushkaghei/Stock-price-prediction/blob/main/Stock_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam
import math
import numpy as np
import random
from collections import deque

#1. Creating the Agent
The Agent code begins with some basic initializations for the various parameters. Some static variables like gamma, epsilon, epsilon_min, and epsilon_decay are defined. These are threshold constant values that are used to drive the entire buying and selling process for stock and keep the parameters in stride. These min and decay values serve like threshold values in the normal distribution.

The agent designs the layered neural network model to take action of either buy, sell, or hold. This kind of action it takes by looking at its previous prediction and also the current environment state. The act method is used to predict the next action to be taken. If the memory gets full, there is another method called expReplay designed to reset the memory.



In [ ]:
class Agent:
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size  # normalized previous days
        self.action_size = 3  # sit, buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

        self.model = load_model("models/" + model_name) if is_eval else self._model()

    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(learning_rate=0.001))

        return model

    def act(self, state):
        if not self.is_eval and random.random() <= self.epsilon:
            return random.randrange(self.action_size)

        options = self.model.predict(state)
        return np.argmax(options[0])

    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])

        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

#2. Define Basic Functions
The formatprice() is written to structure the format of the currency. The getStockDataVec() will bring the stock data into python. Define the sigmoid function as a mathematical calculation. The getState() is coded in such a manner that it gives the current state of the data.

In [ ]:
# prints formatted price
def formatPrice(n):
    return ("-Rs." if n < 0 else "Rs.") + "{0:.2f}".format(abs(n))


# returns the vector containing stock data from a fixed file
def getStockDataVec(key):
    vec = []
    lines = open(key + ".csv", "r").read().splitlines()

    for line in lines[1:]:
        vec.append(float(line.split(",")[4]))

    return vec


# returns the sigmoid
def sigmoid(x):
    return 1 / (1 + math.exp(-x))


# returns an an n-day state representation ending at time t
def getState(data, t, n):
    d = t - n + 1
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1]  # pad with t0
    res = []
    for i in range(n - 1):
        res.append(sigmoid(block[i + 1] - block[i]))

    return np.array([res])

#3. Training the Agent
Depending on the action that is predicted by the model, the buy/sell call adds or subtracts money. It trains via multiple episodes which are the same as epochs in deep learning. The model is then saved subsequently.

In [ ]:
stock_name = input("Enter stock_name: ")
window_size = int(input(" window_size: "))
episode_count = int(input("Episode_count:"))


agent = Agent(window_size)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

for e in range(episode_count + 1):
    print("Episode " + str(e) + "/" + str(episode_count))
    state = getState(data, 0, window_size + 1)

    total_profit = 0
    agent.inventory = []

    for t in range(l):
        action = agent.act(state)

        # sit
        next_state = getState(data, t + 1, window_size + 1)
        reward = 0

        if action == 1:  # buy
            agent.inventory.append(data[t])
            print("Buy: " + formatPrice(data[t]))

        elif action == 2 and len(agent.inventory) > 0:  # sell
            bought_price = agent.inventory.pop(0)
            reward = max(data[t] - bought_price, 0)
            total_profit += data[t] - bought_price
            print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

        done = True if t == l - 1 else False
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state

        if done:
            print("--------------------------------")
            print("Total Profit: " + formatPrice(total_profit))
            print("--------------------------------")

        if len(agent.memory) > batch_size:
            agent.expReplay(batch_size)

    if e % 10 == 0:
        agent.model.save("models/model_ep" + str(e))

Enter stock_name: HDFC
 window_size: 5
Episode_count:2
Episode 0/2
Buy: Rs.51.79
Buy: Rs.49.11
Sell: Rs.50.56 | Profit: -Rs.1.23
Buy: Rs.57.33
Sell: Rs.65.02 | Profit: Rs.15.91
Sell: Rs.57.49 | Profit: Rs.0.17
Buy: Rs.53.90
Buy: Rs.57.05
Buy: Rs.61.09
Buy: Rs.63.37
Buy: Rs.57.28
Buy: Rs.38.46
Sell: Rs.41.83 | Profit: -Rs.12.07
Sell: Rs.45.46 | Profit: -Rs.11.59
Sell: Rs.49.60 | Profit: -Rs.11.49
Sell: Rs.49.96 | Profit: -Rs.13.41
Sell: Rs.69.00 | Profit: Rs.11.72
Buy: Rs.72.26
Sell: Rs.72.10 | Profit: Rs.33.64
Buy: Rs.79.14
Buy: Rs.77.69
Buy: Rs.76.53
Sell: Rs.73.12 | Profit: Rs.0.86
1/1 [==============================] - 0s 25ms/step
Buy: Rs.71.91
1/1 [==============================] - 0s 24ms/step
Buy: Rs.65.47
1/1 [==============================] - 0s 21ms/step
Sell: Rs.68.63 | Profit: -Rs.10.51
1/1 [==============================] - 0s 37ms/step
Sell: Rs.62.17 | Profit: -Rs.15.52
1/1 [==============================] - 0s 24ms/step
Buy: Rs.61.33
1/1 [==============================] 

#4. Evaluation of the model
Once the model has been trained depending on new data, you will be able to test the model for the profit/loss that the model is giving. You can accordingly evaluate the credibility of the model.

In [ ]:
stock_name = input("Enter Stock_name:")
model_name = 'model_ep0'
model = load_model("models/" + model_name)
window_size = model.layers[0].input.shape.as_list()[1]

agent = Agent(window_size, True, model_name)
data = getStockDataVec(stock_name)


l = len(data) - 1
batch_size = 32

state = getState(data, 0, window_size + 1)
total_profit = 0
agent.inventory = []
print(l)
for t in range(l):
    action = agent.act(state)

    # sit
    next_state = getState(data, t + 1, window_size + 1)
    reward = 0

    if action == 1:  # buy
        agent.inventory.append(data[t])
        print("Buy: " + formatPrice(data[t]))

    elif action == 2 and len(agent.inventory) > 0:  # sell
        bought_price = agent.inventory.pop(0)
        reward = max(data[t] - bought_price, 0)
        total_profit += data[t] - bought_price
        print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

    done = True if t == l - 1 else False
    agent.memory.append((state, action, reward, next_state, done))
    state = next_state

    if done:
        print("--------------------------------")
        print(stock_name + " Total Profit: " + formatPrice(total_profit))
        print("--------------------------------")

Enter Stock_name:hdfc
262
1/1 [==============================] - 0s 82ms/step
Buy: Rs.46.03
1/1 [==============================] - 0s 33ms/step
Buy: Rs.47.67
1/1 [==============================] - 0s 31ms/step
Buy: Rs.50.71
1/1 [==============================] - 0s 30ms/step
Buy: Rs.50.22
1/1 [==============================] - 0s 29ms/step
Buy: Rs.50.20
1/1 [==============================] - 0s 29ms/step
Buy: Rs.51.67
1/1 [==============================] - 0s 36ms/step
Buy: Rs.49.53
1/1 [==============================] - 0s 62ms/step
Buy: Rs.50.15
1/1 [==============================] - 0s 36ms/step
Buy: Rs.50.12
1/1 [==============================] - 0s 28ms/step
Buy: Rs.50.22
1/1 [==============================] - 0s 21ms/step
Buy: Rs.51.99
1/1 [==============================] - 0s 21ms/step
Buy: Rs.55.99
1/1 [==============================] - 0s 21ms/step
Buy: Rs.57.96
1/1 [==============================] - 0s 23ms/step
Buy: Rs.56.72
1/1 [==============================] - 0s 27ms/ste